In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

## Load data

In [2]:
data = pd.read_csv('abalone_dataset.csv')
sample = pd.read_csv('abalone_app.csv')
data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
0,M,0.535,0.420,0.150,0.6995,0.2575,0.1530,0.2400,3
1,I,0.510,0.380,0.115,0.5155,0.2150,0.1135,0.1660,1
2,I,0.185,0.130,0.045,0.0290,0.0120,0.0075,0.0095,1
3,M,0.550,0.450,0.170,0.8100,0.3170,0.1570,0.2200,3
4,I,0.535,0.415,0.150,0.5765,0.3595,0.1350,0.2250,1


## Transform

In [3]:
def conv(sex):
    if sex == 'M':
        sex = 0
    elif sex == 'F':
        sex = 1
    else:
        sex = 2
        
    return sex

data['sex'] = data['sex'].apply(conv)
sample['sex'] = sample['sex'].apply(conv)
data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,type
0,0,0.535,0.420,0.150,0.6995,0.2575,0.1530,0.2400,3
1,2,0.510,0.380,0.115,0.5155,0.2150,0.1135,0.1660,1
2,2,0.185,0.130,0.045,0.0290,0.0120,0.0075,0.0095,1
3,0,0.550,0.450,0.170,0.8100,0.3170,0.1570,0.2200,3
4,2,0.535,0.415,0.150,0.5765,0.3595,0.1350,0.2250,1


## Separating

In [4]:
features = data.columns.difference(['type'])

X = data[features]
y = data['type']
samp = sample

## Creating and training a classifier

In [13]:
knn = KNeighborsClassifier(n_neighbors = 1,
                           weights = 'distance',
                           algorithm = 'auto',
                           p = 2)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='distance')

In [14]:
X_new = np.array([[0, 0.535, 0.420, 0.150, 0.6995, 0.2575, 0.1530, 0.2400]])
X_new.shape
knn.predict(X_new)

array([2])

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {
            "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
            "n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            "p": [1, 2],
            "weights": ['distance', 'uniform']
}
grid_search = GridSearchCV(knn, param_grid, scoring="accuracy")
grid_search.fit(X, y)

knn = grid_search.best_estimator_ 
grid_search.best_params_, grid_search.best_score_

/home/franklin/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


({'algorithm': 'auto', 'n_neighbors': 10, 'p': 2, 'weights': 'distance'},
 0.6344189016602809)

In [26]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(knn, X, y, scoring='accuracy', cv=5)
print(scores.mean())

0.6388894133422729


In [ ]:
import requests

#realizando previsões com o arquivo de
print(' - Aplicando modelo e enviando para o servidor')
y_pred = knn.predict(sample)

# Enviando previsões realizadas com o modelo para o servidor
URL = "https://aydanomachado.com/mlclass/03_Validation.php"

#TODO Substituir pela sua chave aqui
DEV_KEY = "VovóLearn"

# json para ser enviado para o servidor
data = {'dev_key':DEV_KEY,
        'predictions':pd.Series(y_pred).to_json(orient='values')}

# Enviando requisição e salvando o objeto resposta
#r = requests.post(url = URL, data = data)

# Extraindo e imprimindo o texto da resposta
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")